In [19]:
# 대한민국 구석구석 사이트 상세 정보 수집하기

#Step 1. 필요한 모듈을 로딩합니다
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          
import pandas as pd    
import math

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 연습문제: 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.")
print("=" *100)

query_txt = input('1.정보를 수집할 키워드는 무엇입니까?: ')

cnt = int(input('2.몇 건의 정보를 수집할까요? :'))
page_cnt = math.ceil( cnt / 10 )
print('총 %s 건의 정보를 수집하기 위해 %s 페이지까지 이동할 예정입니다' %(cnt , page_cnt))

#Step 3. 수집된 데이터를 저장할 폴더 이름 입력받기 
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(기본값:c:\\py_temp\\):")
if f_dir == '' :
    f_dir="c:\\py_temp\\"
    
print("\n")    

#Step 3. 검색어 입력한 후 검색하여 해당 장르로 이동하기
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

query_url = 'https://korean.visitkorea.or.kr/'

driver.get(query_url)
time.sleep(2)
driver.maximize_window()

s_time = time.time( )
element = driver.find_element(By.ID,"inp_search")
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(2)

# Step 5. 본문 내역 추출하기
time.sleep(0.2)
s_time = time.time( ) 

no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
org2=[]          # 지자체명 컬럼
contents2=[ ]     # 본문 내용 컬럼
tag2=[ ]         # 해시태그 컬럼

no = 1           # 게시글 번호 초기값

for a in range(1,page_cnt+1) :
    print('')
    print('현재 %s 페이지에 있는 정보를 수집합니다' %a)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType type1').find_all('li')
    cnt_no = 1
    
    for b in content_list:
        
        try :
            title = b.find('div','area_txt').find('div','tit').get_text().strip()
        except :
            continue
        else :
            # 1. 게시글 번호
            print("\n")
            print("%s 번째 정보를 추출하고 있습니다============" %no)
            no2.append(no)
            print("1.번호 : %s" %no)

            # 2. 게시글 제목
            title2.append(title)
            print("2.제목 : %s" %title )
            
            # 3. 지자체이름
            org = b.find('div','service').find('p').get_text().strip()
            org2.append(org)
            print("3.지자체이름: %s" %org)
            
            #4. 해시태그
            try :
                tag = b.find('p','tag_type').get_text().replace("#"," #")
            except :
                tag = ' '
                
            tag2.append(tag)
            print("4.해시태그: %s" %tag)
            
            #5. 본문   
            # 1 페이지에만 광고가 들어 있어서 xpath 값이 다름
            # 2 페이지부터는 광고가 없어서 1씩 증가시키면 됨
            if a == 1 and cnt_no == 5 :
                cnt_no += 1
                
            driver.find_element(By.XPATH,'//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a' %cnt_no).click()
            cnt_no += 1
            
            time.sleep(3)
            
            html_2 = driver.page_source
            soup_2 = BeautifulSoup(html_2, 'html.parser')
            
            content1 = soup_2.find('div','inr_wrap')
            content2 = soup_2.find('div','box_txtPhoto')
            
            if content1 :
                content = content1.find('div','inr').find('p').get_text()
                contents2.append(content)
                print('5.본문내용:', content)
            elif content2 :
                content_1 = soup_2.find('div','box_txtPhoto')
                try :
                    content_1.style.decompose()
                    content_1.script.decompose()
                except :
                    content_2 = content_1.find_all('div','txt_p')
                    print('5.본문내용:') 
                    for b in content_2 :
                        content = b.get_text().strip()
                        print(content)
                        contents2.append(content)
                else :
                    content_2 = content_1.find_all('div','txt_p')
                    print('5.본문내용:') 
                    for b in content_2 :
                        content = b.get_text().strip()
                        print(content)
                        contents2.append(content)
                           
            driver.back()
            
            time.sleep(5)
                       
            no += 1         # 전체 게시글 번호용 값
            
            if no > cnt :
                 break
                
            time.sleep(0.5)
            
    a += 1
    try :
        driver.find_element(By.LINK_TEXT,'%s' %a).click()
    except :
        driver.find_element(By.LINK_TEXT,'다음').click()

    time.sleep(0.5)

# Step 6. 결과 저장하기
import time
import os

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt)

fc_name = f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt+'\\'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.csv'
fx_name = f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt+'\\'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.xls'

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['지자체명']=pd.Series(org2)
df['본문내용']=pd.Series(contents2)
df['해시태그']=pd.Series(tag2)

# xls , csv 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8", engine='openpyxl')
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print('xls 저장경로: ',fx_name)
print('csv 저장경로: ',fc_name)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("=" *80)

driver.close( )

 연습문제: 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.
1.정보를 수집할 키워드는 무엇입니까?: 제주도여행
2.몇 건의 정보를 수집할까요? :123
총 123 건의 정보를 수집하기 위해 13 페이지까지 이동할 예정입니다
3.파일을 저장할 폴더명만 쓰세요(기본값:c:\py_temp\):



현재 1 페이지에 있는 정보를 수집합니다


1 번째 정보를 추출하고 있습니다============
1.번호 : 1
2.제목 : 제주도 서쪽에서 즐기는 꽉 찬 여행 코스
3.지자체이름: 제주도
4.해시태그:  #제주도여행 #제주도가볼만한곳 #도두봉 #도두동무지개해안도로 #무지개해안도로 #구엄포구 #엄장해안길 #드라이브코스 #오설록 #오설록티뮤지엄 #제주도 
5.본문내용:
볼 것도, 즐길 것도 너~무 많은 제주도! 시간이 한정되어 있다면, 알짜배기 여행지만 쏙쏙 골라 떠나야겠죠?​오늘은 제주도 서쪽을 따라 떠나는 꽉 찬 여행 코스를 소개해드립니다.
첫 번째 여행지는 바로 도두봉. 공항과 가까우며 탁 트인 풍경을 볼 수 있는 곳입니다. 도두봉 옆, 최근 인기가 많은 무지개해안도로도 함께 소개합니다. 우선 도두봉은 도두동 해안가에 있습니다.
해발이 63m 정도로 높지 않고, 도두봉 정상부분에는 키세스 초콜릿 모양의 나무 터널이 있는데요. SNS 인생샷 명소로 사랑받는 장소입니다.​올라가는 등산로는 계단으로 이루어져 있습니다. 대략 10분 정도 소요되며, 정상에서 바다와 한라산을 볼 수 있어 가볍게 올라 풍경을 조망하기 좋습니다.
내려 온 뒤 바로 옆에 있는 무지개 해안도로를 꼭 들러 보는 것을 추천해요! 1km 남짓한 해안도로를 배경으로 빨주노초파남보 무지개색으로 칠해진 방호벽을 볼 수 있습니다. ​약 15분에서 20분 사이의 거리로 바다를 보며 산책하고 사진을 찍기에 제격입니다.
※ 도두봉     -위치 : 제주 제주시 도두일동 산1​​          ※ 도두동무지개해안도로     -위치 : 제주 제주시 도두일동 1734
구엄포구에 있는 엄장해안길은 구



4 번째 정보를 추출하고 있습니다============
1.번호 : 4
2.제목 : 효리네민박 촬영지부터 매화꽃 명소까지, ♥제주 봄여행 코스♥
3.지자체이름: 제주도
4.해시태그:  #제주3월가볼만한곳 #제주3월여행코스 #제주도3월여행 #효리네민박촬영지 #효리네민박여행 #제주도여행코스 #제주도봄여행추천코스 #제주봄에가볼만한곳 #제주매화꽃여행 #제주여행일정추천 #봄꽃놀이 #봄꽃여행 #TV_속_여행지 #TV속촬영지여행 #영화&드라마&예능_속_여행지 #미리봄_여행지 
5.본문내용:
● 너븐숭이 4.3 기념관
효리네민박 손님이 찾은 제주4.3사건을 간직한 기념관. 우리의 잊어서는 안될 역사의 한부분이지요. 북촌리는 4.3역사의 모든 것이라고 할 만큼 아픔이 있는 곳입니다. 제주시 조천읍의 동쪽 끝에 자리잡은 해변마을. 일제시대에는 항일운동가가 많았고 해방 후에는 인민위원회를 중심으로 자치조직이 활성화 됐던 곳이기도 합니다. 너븐숭이 4.3기념관 내에는 전시관, 탐구관, 묵상의 방이 있고 밖으로는 위령비와 각명비, ‘순이삼촌비;와 방사탑이 있어요. 아직 많이 알려지지 않아 전시관은 늘 한가한 마을 처럼 조용하기만 합니다.      -전화 : 064-783-4303     -주소 : 제주시 조천읍 북촌3길 3     -운영시간 : 09:00 ~ 18:00 / 매월 둘째, 넷째 월요일 휴관        -관람료 : 무료
● 갯깍주상절리
제주도 하면 서귀포에 있는 주상절리대가 유명하죠. 주상절리 사이로 파도가 맞부딪치면서 몇 미터나 바닷물이 튀어 올라옵니다. 주상절리 중 으뜸으로 치는 갯깍 주상절리가 효리네민박 1시즌에 나왔었죠. 반딧불이 보호구역과 연결되는 해안 주상절리는 1km에 이른다고 합니다. 조근모살해수욕장과 어울려 몽돌 가득한 해안을 주차장에서 10여 분 걸어가면 주상절리와 함께 다람쥐굴이라 불리는 작은 해식동굴이 있어요. 입구부터 아찔한 아치형으로 신비로운 모양으로 이루어져 있고 내부를 조각하듯 주상절리의 단면은 제주도의 또다른 새로움을 가져다 줍니다. 일



7 번째 정보를 추출하고 있습니다============
1.번호 : 7
2.제목 : [국내 트레킹 추천] 제주 올레길 걷기 여행, 준비부터 코스 선택까지 꿀팁 총정리!
3.지자체이름: 제주도
4.해시태그:  #제주올레길 #트레킹코스 #트레킹 #제주가볼만한곳 #제주여행 #걷기여행 
5.본문내용:
제주하면 떠오르는 그것. ‘올레’라는 이름입니다. 올레라는 단어가 누군가에게는 제주도를 한 바퀴 도는 걷기 여행 코스일 수도 있고, 누군가에게는 소주의 이름일 수도 있습니다. ‘올레’는 큰길에서 집으로 들어가는 길목에 있는 작은 골목길을 뜻하는 제주도 방언입니다. 집 안쪽으로 들어서며 볼 수 있는 검은빛 현무암을 쌓아 만든 돌담길이 굽이굽이 이어지는 모습을 상상하셨나요? 맞습니다. 바로 그 골목길이 올레입니다.
제주올레는 제주도 마을의 골목길에서 제주를 하나로 크게 잇는 길이 되었습니다. 총 스물여섯 개의 코스가 촘촘히 연결되어 있어 제주의 속살을 하나씩 감상할 수 있는 길인데요. 무려 425km에 달하는 길이이니만큼 쉽게 도전하기 어려운 것도 사실입니다.
그러나 꼭 완주할 필요가 있을까요. 올레길이 제주의 숨겨진 아름다움을 곳곳에 품고 있는데 말입니다. 하나씩, 천천히 시작해 보세요. 어느새 제주올레의 매력에 흠뻑 빠지실 겁니다.
본격적인 제주올레 걷기를 준비해볼까요? 우선 준비물입니다. 제주올레를 걷기 위해서는 운동화 혹은 트레킹화, 가벼운 배낭과 충분한 물, 챙 달린 모자와 선글라스 등 기본적으로 야외 활동하기 좋은 복장이 좋습니다. 주로 평평한 길을 걷게 되지만, 때로는 숲길도, 때로는 자갈길도 지납니다.
제주올레 가이드북은 제주올레를 한층 더 깊이 있게 이해하며 걸을 수 있는 교과서입니다. 각 코스에 대한 설명과 지도는 물론, 깊이 있는 이야기들이 알차게 수록되어 있으니 길을 걷기 전 꼭 한 번 읽어보세요.
제주올레 패스포트는 올레길을 더 즐겁게 걸을 수 있게 해주는 아이템입니다. 시작점과 종점, 중간 지점에 각각 하나씩 설치된 스탬프를 찍어 패스포트를 완성



9 번째 정보를 추출하고 있습니다============
1.번호 : 9
2.제목 : 알록달록 가을빛 제주
3.지자체이름: 제주도
4.해시태그:  #장애인추천코스 #장애인추천여행지 #공공누리 #무장애여행지 #열린관광_모두의여행 #제주도 
5.본문내용:
<전체이동경로>제주해녀박물관  → 자동차 35분 28.3㎞ → 제주돌문화공원 → 자동차 9분 6.1㎞ → 붉은오름자연휴양림 → 자동차 2분 1.2㎞ → 사려니숲길 → 자동차 28분 25.2㎞ → 서귀포 치유의숲
제주는 늘 새롭다. 계절마다 뿜어내는 색이 다채롭고, 날씨에 따라 풍경의 결이 달라진다. 봄이면 노오란 유채꽃이 푸른 바다와 어우러지고, 여름이면 분홍색, 하늘색 수국이 길을 안내한다. 가을에는 하얀 억새가 높은 하늘로 손짓하고, 겨울이면 눈 쌓인 한라산 꼭대기가 늠름하다. 사방이 바다로 둘러싸인 섬, 368개의 오름을 품은 제주의 구석구석은 언제나 사랑스럽다. 올록볼록하게 튀어나온 오름을 배경으로 독특한 멋을 풍기는 해안도로에서부터 청량한 숲의 기운을 뿜어내는 휴양림과 숲길을 찾아가 보자.
alt=“제주 중심 주요 관광지를 점으로 표현한 지도”ir부분제주국제공항해녀박물관제주돌문화공원붉은오름자연휴양림사려니숲 무장애나눔길서귀포 치유의숲서귀포시
제주해녀박물관은 다른 어느 지역에서도 만날 수 없는, 오롯이 제주에서만 만날 수 있는 박물관이라는 점에서 특별하다. 제주시에서 동쪽으로 차를 타고 1시간, 세화리에 위치한 해녀박물관에서는 제주 해녀의 삶을 엿볼 수 있다.해녀박물관 1층으로 입장하면 해녀의 집을 통째로 옮겨둔 것 같은 전시장이 펼쳐진다. 제주 해녀들의 소박한 살림살이를 엿볼 수 있는 옷과 생활 도구들이 전시되어 있고, 제주도 토속 음식들도 모형으로 만난다. 전시관 2층에는 해녀들이 바닷가에서 몸을 말리는 불턱을 재현해두고, 해녀들의 옷과 장비를 전시했다. 제주의 해녀들이 실제로 물질을 하는 영상이 불턱 뒤로 흐른다. 전시장 곳곳에서 만나는 짧은 영상을 감상하다 보면, 어머니 같고, 할머니 같은 해녀들의



10 번째 정보를 추출하고 있습니다============
1.번호 : 10
2.제목 : [제주] 여행자를 위한 안성맞춤 숙소, 제주 아트스테이 서귀포 하버
3.지자체이름: 제주도 서귀포시
4.해시태그:  #여행Q레이터 #여행자의방 #한국관광품질인증 #추천숙박 #아트스테이서귀포하버 #제주도 #서귀포 #여행Q레이터체험기 #품질인증콘텐츠 
5.본문내용:
서귀포 중심부에 위치한 아트스테이 서귀포 하버는 제주 올레길과 제주 환상자전거 코스에서 매우 가까워 자전거족과 뚜벅이족이 머물기에 좋은 숙소다. 인근에 버스정류장이 있어 서귀포를 방문하는 일반 관광객들 또한 접근하기 좋다. 제주국제공항에서 출발 시 600번 버스를 이용하면 환승 없이 한 번에 아트스테이까지 갈 수 있다. 숙소 내 승용차 기준 최대 98대까지 수용 가능한 넓은 주차시설을 보유하고 있어 자가용 이용 시에도 큰 문제가 없다.
2018년 11월 오픈한 신축 건물로 내외관이 모두 깔끔하다. 건물 내 이동 통로 또한 심플하게 구성되어 있어 길을 헤매는 일이 없다. 문이 열리면 바로 보이는 로비에서 체크인 후 정면에 있는 엘리베이터를 이용해 객실로 이동하면 된다. 입구 오른편에 보이는 로비는 넓고 깔끔해서 대화를 나누거나 휴식하기 좋다. 로비에는 컴퓨터 2대가 배치되어 있어 무료로 인터넷을 사용할 수 있고, 전자 다트와 같은 오락기기도 설치되어 있다.
아트스테이에는 총 5가지 종류의 객실이 있다. 스탠더드, 디럭스, 프리미어, 주니어스위츠, 로열 스위트가 있으며 객실별로 더블, 트윈, 온돌 중 하나를 고를 수 있다. 객실에는 에어컨, 냉장고, 드라이어, 간단한 취사도구, 세면용품, 평면TV, 금고가 구비되어 있다. 아트스테이의 가장 큰 특징은 프리미어 객실 중 반려동물과 함께 묶을 수 있는 펫 객실이 있는 것이다. 객실 예약 시 오션뷰와 시티뷰 혹은 하프 오션뷰를 선택할 수 있으며 가장 인기가 많은 것은 서귀포의 바다가 훤히 보이는 오션뷰 객실이다. 피트니스센터는 무료다.
특별한 두 가지 서비스아트스테이 서귀

...



99 번째 정보를 추출하고 있습니다============
1.번호 : 99
2.제목 : 구엄포구
3.지자체이름: 제주도 제주시
4.해시태그:  #관광지 #제주여행 #구엄리 #제주도이색여행지 #일몰 #구엄포구 #제주도여행지추천 #제주도해안도로 #제주도애월 #구엄리돌염전 #애월여행지 
5.본문내용: 노을이 아름다운 애월 해안도로를 따라가다 보면 작고 아담한 구엄포구를 만난다. 구엄포구 바로 옆에는 제주의 옛 생활을 엿볼 수 있는 ''돌염전''이 있다. 구엄리 ''돌염전''은 넓게 펼쳐진 현무암 위에서 소금을 생산하던 곳이다. 이곳은 ''소금빌레''라고도 일컬어지는 곳으로, 구엄리 주민들이 천연 암반지대에서 소금을 생산하던 염전이다. 여기서 ‘빌레’란 너럭바위란 뜻의 제주도 사투리로, 즉 돌소금 밭이라는 뜻이다. ''돌염전''이 있는 해안길은 국내의 아름다운 해안도로인 ''해안 누리길''로 선정된 바 있다. 구엄포구 한편에는 돌로 만든 도대불도 있다. ‘도대불’이란 어부가 밤중에 고기잡이를 마치고 집으로 돌아올 때 불을 밝혀 안전하게 길잡이 역할을 했던 등대다. ‘도대불’과 ‘소금빌레’ 모두 제주에서만 볼 수 있는 중요한 해양 문화유산이다.


100 번째 정보를 추출하고 있습니다============
1.번호 : 100
2.제목 : 해외여행 그리움을 달래줄  서울 근교 풀빌라
3.지자체이름: 경기도
4.해시태그:  #경기도가볼만한곳 #풀빌라 #안산풀빌라 #파주풀빌라 #이천풀빌라 #풀빌라펜션 #독채펜션 #세시간전 
5.본문내용:
에디터는 따뜻해지는 날씨를 체감하면서 해외여행을 못 가는 서러움이 더 커지는 중이다. 그래서 준비했다! 멀리 못 간다면 가까운 데라도 떠나자. 여행 욕구를 채워줄 서울 근교 풀빌라 펜션을 소개한다. 감성적인 객실과 물놀이까지! 머무는 동안 1시간이 1초처럼 느껴질지도 모른다.
이름처럼 바다를 원 없이 눈에 담을 수 있는 여기바다. 독특한 건축 디자인과 감각적인 스타일링을 자랑하는 곳이다. 건물 외관이 마치 이모티콘 같기도 하다.🏠@chi


현재 11 페이지에 있는 정보를 수집합니다


101 번째 정보를 추출하고 있습니다============
1.번호 : 101
2.제목 : 제주도 관광특구
3.지자체이름: 제주도
4.해시태그:  #관광지 #가족과함께 #연인과함께 #자연환경 #자연관광지 #제주관광특구 #제주여행 
5.본문내용: 한반도의 남서해상에 위치하는 제주도는 특별자치도로 지정된 국내 최대의 관광 지역이다. 전 지역이 관광특구로 지정될 만큼 발길 닿는 곳마다 아름다운 자연 경관이 펼쳐진다. 내외국인을 합쳐 한 해 약 1,500만명 이상의 관광객이 다녀갈 정도로 매력적인 이 섬의 크기는 남북으로 31km, 동서로 73km에 이른다. 유네스코 3관왕(세계자연유산, 세계지질공원, 생물권보전지역) 지역이며, 세계7대자연경관 지역으로 지정된 제주도는 온대 지방의 특징을 가진 한반도의 육지와는 다른 기후대에 자리했다. 덕분에 아열대에서 아한대에 이르는 제주만의 식생을 보유했다.제주도의 색다른 자연은 이 섬을 찾은 여행자들의 발길을 붙잡는다. 섬 한가운데 우뚝 솟은 한라산과 산기슭에 솟아 오른 크고 작은 오름이 만들어 내는 풍경, 해안에 펼쳐진 주상절리는 화산섬 제주도를 대표하는 풍경이다. 바다에서 한라산을 향해 오르다 만나는 중산간 지역은 자연이 주는 선물이다. 제주세계자연유산센터가 있는 거문오름을 찾아 중산간의 자연을 탐방하는 것도 좋다. 제주의 방언과 생활풍습도 이색적이다. 해안선을 따라 이어진 182km의 일주도로는 자동차는 물론 스쿠터나 자전거를 타고 달리면 좋다. 도보길인 ‘제주올레’를 따라가면 걸어서 제주도를 한 바퀴 돌 수 있다.제주도는 ‘국제관광도시’ ‘국제자유도시’ 라는 이름에 걸맞게 관광은 물론 정치·경제에서도 육지의 자치구들보다 자유롭고 다양한 정책을 펼치고 있어 외국인들에게도 인기 있다.* 제주도 관광특구 현황1) 범위 : 제주특별자치도 전역(부속 도서 제외)2) 면적 : 1,809.56k㎡3) 관광특구 지정일 : 1994년 8월 31일* 제주도 관광특구 시설 현황한라산, 오름, 



104 번째 정보를 추출하고 있습니다============
1.번호 : 104
2.제목 : 곰막식당/곰막
3.지자체이름: 제주도 제주시
4.해시태그:  #음식 #제주여행 #제주도맛집추천 #제주도맛집 #제주가볼만한곳 #제주도성게국수 #제주도여행 #제주곰막식당 #제주맛집 
5.본문내용: 전복죽 전문점인 곰막식당의 깨가 듬뿍 들어간 전복죽은 첫술부터 진한 고소함을 느낄 수 있다. 깊은 바다향이 나는 성게 국수와 회국수 역시 이 집의 대표 메뉴이며 비린내 없이 담백하다. 이곳은 생면으로 국수를 만들어 쫄면같이 쫄깃쫄깃한 식감이 입맛을 더 돋운다. 활어회도 인기 메뉴로 매일 지하 80m에서 염지하수를 끌어올려 사용하기 때문에 더욱더  싱싱한 회를 먹을 수 있다. 인기 많은 맛집답게 기본 반찬은 제공하지만, 이후에는 주류, 음료, 추가 반찬, 물, 물티슈, 국물, 앞치마 등을 셀프로 가져와야 한다.


105 번째 정보를 추출하고 있습니다============
1.번호 : 105
2.제목 : 도두해녀의집
3.지자체이름: 제주도 제주시
4.해시태그:  #음식 #제주여행 #제주도맛집 #제주맛집 #제주가볼만한곳 #제주공항근처맛집 #제주도여행 #도두해녀의집 #제주도맛집추천 
5.본문내용: 도두해녀의집은 제주 국제공항에서 10분 거리에 있어 제주를 오갈 때 들르기 좋다. 특 물회, 전복물회, 한치물회 등 다양한 물회 메뉴가 있으며 계절별로 신선한 재료가 준비되니 방문하기 전에 주문 가능한 메뉴를 확인 후 방문하면 좋다. 성게비빔밥의 경우 하루 한정 수량이 있어 점심시간 전에도 매진된다고 하니 너무 늦지 않게 방문하는 게 좋다. 도두해녀의집은 물회 맛집으로 알려졌지만 전복죽 맛집으로도 유명하다. 전복 내장 특유의 고소한 맛과 진한 맛이 있어 이곳을 방문한 고객들은 인생 전복죽이라고 평가하기도 한다.


106 번째 정보를 추출하고 있습니다============
1.번호 : 106
2.제목 : 예래포구(하예포구)
3.지자체이름: 제주도 서귀포시
4.해시태그:  #관광지 #제주여행



116 번째 정보를 추출하고 있습니다============
1.번호 : 116
2.제목 : 대록산 (큰사슴이오름)
3.지자체이름: 제주도 서귀포시
4.해시태그:  #관광지 #제주도여행 #대록산 #제주오름 #제주도여행추천 #제주가볼만한곳 #제주도억새밭 #제주관광명소 
5.본문내용: 대록산은 서귀포 표선면 가시리에 있는 고도 466m의 측화산(기생화산)이다. 전반적으로 가파르고 둥근 모습이 사슴 모양을 닮았다고 하여 큰사슴이오름으로 불린다. 정상의 봉우리 사이에는 둥그렇게 패어있는 원형 화구를 가지고 있다. 사시사철 자연의 아름다움을 자랑하는 곳이지만, 특히 가을이면 넓게 펼쳐진 억새로 많은 관광객이 찾는다. 정상으로 가는 길목에 있는 이 억새밭은 넓은 들판과 대록산을 배경으로 완벽한 포토존을 만들어 낸다. 억새밭에서 정상까지는 약 1.3km 정도이며, 다소 계단이 많으니 편안한 복장으로 다녀오는 것을 추천한다.


117 번째 정보를 추출하고 있습니다============
1.번호 : 117
2.제목 : [랜선가이드] 1000칼로리 태우는 여행법이 궁금하다면? '러닝여신' 안정은과 함께 렛츠고 런트립!
3.지자체이름: 전국
4.해시태그:  #랜선가이드 #러닝전도사 #런트립 #요즘_대세는_런트립 
5.본문내용:
“여행작가가 아니라도 괜찮아!”                     [랜선가이드]는 우리 주위의 ‘여.잘.알.’(여행을 잘 아는 사람)을 만나 그들의 경험담을 듣거나 다양한 여행지를 추천받는 인터뷰 기사입니다. 생생한 정보로 당신의 여행을 가이드 해드립니다.
● 이달의 가이드: 달리는 여신, 러닝 전도사 안정은
prologue                 안정은은 하루 30분 달리기로 새로운 인생을 찾은 러너입니다. 달리기를 하며 취업 스트레스를 풀다가 본격적으로 러너의 길에 들어선 겁니다. 각종 마라톤 풀코스와 트레일 러닝, 트라이애슬론을 완주하고, 그 경력을 바탕으로 달리기의 매력을 알리는데 앞장서고 있습니다. 런트립 기획, 모델, 강연, 코치, 저



122 번째 정보를 추출하고 있습니다============
1.번호 : 122
2.제목 : 바리메 / 바리메오름
3.지자체이름: 제주도 제주시
4.해시태그:  #관광지 #제주여행 #바리메오름 #제주관광명소 #제주도여행 #제주가볼만한곳 #제주도등산로 #제주도여행추천 
5.본문내용: 바리메 오름은 제주시 애월읍 어음리에 있는 오름으로 해발 763m이며 오름 중 가장 높고 경사가 가파른 오름에 속한다. 오름 정상 분화구 모양이 절에서 쓰는 승려들의 공양 그릇 바리를 닮았다고 하여 바리매 오름으로 불리게 되었다 정상 분화구 깊이는 78m이고 직경 130m이다. 오름의 서쪽으로는 목장이 들어서 있고, 나머지 경사면은 소나무가 무성히 자라고 있다. 경사가 있는 오름으로 둘레길처럼 운동화 차림보다는 등산화 차림이 편할 수 있다. 등산로를 따라 다양한 야생화와 피어있고 정상에는 다래 오름, 폭낭오름, 괴오름, 북돌아진 오름 등이 있어 크고 작은 오름들이 어울려 있는 진풍경을 만날 수 있다. 산악자전거 코스로도 인기다.


123 번째 정보를 추출하고 있습니다============
1.번호 : 123
2.제목 : 협재해녀의집
3.지자체이름: 제주도 제주시
4.해시태그:  #음식 #제주여행 #제주해물라면 #제주도맛집추천 #제주도맛집 #협재해녀의집 #제주도여행 #협재맛집 #제주맛집 
5.본문내용: 해물라면에 돌문어 숙회, 뿔소라 회, 그리고 전복·소라·문어숙회·멍게·새우가 들어간 모둠 해산물의 조화로운 맛을 즐길 수 있다. 해산물이 가득 들어 있는 라면은 국물이 시원하고 얼큰하며 양도 푸짐하다. 파 송송 콩나물 팍팍 들어가 술안주 또는 해장용으로도 좋다. 직접 잡아 온 재료로 만들어주기 때문에 더욱 신선한 해산물을 즐길 수 있으며 바 형태로 되어 있는 실내는 비양도를 바라보며 식사하기에도 좋고 혼밥하기에도 최적이다. 주문과 결제는 셀프로 진행되는데 현금결제 시 맞은편 GS25에서 아메리카노를 무료로 제공한다.


크롤링을 요청한 총 123 건 중에서 123 건의 데이터를 수집

In [17]:
print(cnt_no)

11
